In [1]:
#29/04/2022
#=====================================
# Yolo V-3 by irfanhasib.me@gmail.com
#=====================================
import os
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['PYTHONHASHSEED']=str(0)
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import cv2
import numpy as np
import time
import glob
import pandas as pd
from matplotlib import pyplot as plt
from multiprocessing import Process, Queue, Pipe
import time
import shutil
import json
from tqdm import tqdm,trange
import pickle
import zlib
from datetime import datetime

from yolo.model import YoloModel, calc_yolo_loss, calc_seg_loss
from yolo.decoder import YoloDecodeNetout
from yolo.dataset import Dataset
from yolo.eval import get_mAP
from yolo.utils import Utils
from yolo.seg_loader import Seg_Utils
from yolo.config import *
from yolo.tf import *

2022-10-11 09:33:41.797315: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


GPUs [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2022-10-11 09:33:42.655059: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-10-11 09:33:42.655724: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-10-11 09:33:42.686730: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-11 09:33:42.687106: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1660 Ti computeCapability: 7.5
coreClock: 1.59GHz coreCount: 24 deviceMemorySize: 5.80GiB deviceMemoryBandwidth: 268.26GiB/s
2022-10-11 09:33:42.687150: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-10-11 09:33:42.694048: I tensorflow/stream_executor/platform/

In [2]:
if not os.path.exists(TRAIN_CHECKPOINTS_FOLDER): os.makedirs(TRAIN_CHECKPOINTS_FOLDER)
with open(TRAIN_CHECKPOINTS_FOLDER +'/params.txt','w') as file:
    log_str='Time '.ljust(30)+': '+str(datetime.now())+'\n'
    for key in list(params.keys())[:-1]:
        if key[:2] != '__':
            log_str += key.ljust(30)+': ' + str(params[key])+'\n'
    print(log_str)
    file.write(log_str)

        
curr_time=time.time()
print('System time : ',curr_time)
os.system(f"cp yolo-v3.ipynb {TRAIN_CHECKPOINTS_FOLDER}/yolo-v3_{str(curr_time)}.ipynb")
if not os.path.exists(TRAIN_CHECKPOINTS_FOLDER+'/yolo/'): os.makedirs(TRAIN_CHECKPOINTS_FOLDER+'/yolo/')
files = glob.glob('yolo/*')
for file in files:
    try : os.system(f"cp -r {file} {TRAIN_CHECKPOINTS_FOLDER}/{file}")
    except PermissionError:
        print('PermissionError : ',file)

Time                          : 2022-10-11 09:33:43.154499
DEBUG_MODE                    : False
RAW_DATA_DIR                  : /home/irfan/Desktop/Code/Datasets/
DATA_DIR                      : /home/irfan/Desktop/Data/
TRAIN_CHECKPOINTS_FOLDER      : logs/exp-MNET_V2_224_MUL_SIG_5110
YOLO_TYPE                     : yolov3
YOLO_MODEL                    : mobilenet
YOLO_MODEL_LOAD_WTS           : model_data/mobilenet_v2_1.0_224_mod.h5
YOLO_FRAMEWORK                : tf
YOLO_V3_WEIGHTS               : model_data/yolov3.weights
YOLO_V4_WEIGHTS               : model_data/yolov4.weights
YOLO_V3_TINY_WEIGHTS          : model_data/yolov3-tiny.weights
YOLO_V4_TINY_WEIGHTS          : model_data/yolov4-tiny.weights
YOLO_TRT_QUANTIZE_MODE        : INT8
YOLO_CUSTOM_WEIGHTS           : True
YOLO_COCO_CLASSES             : model_data/coco/coco.names
YOLO_STRIDES                  : [16, 32, 64]
YOLO_ANCHOR_PER_SCALE         : 3
YOLO_MAX_BBOX_PER_SCALE       : 100
YOLO_INPUT_SIZE               : 224

cp: cannot stat 'yolo-v3.ipynb': No such file or directory


In [3]:
class loss_dict_obj(dict):
        def tf2np(self,val):
            if hasattr(val,'numpy'):
                val=val.numpy()
            else:
                if val==None: val=0
            return val

        def sum_update(self,c_dict):
            for key,val in c_dict.items():
                if key in list(self.keys()):
                    self[key]+=self.tf2np(c_dict[key])
                else:
                    self[key]=self.tf2np(c_dict[key])


        def ext_update(self,c_dict,_ext='_ext'):
            for key,val in c_dict.items():
                     self[_ext+key]=self.tf2np(val)

        def divide(self,div_val):
            for key,val in self.items():
                if type(div_val)==dict or type(div_val)==loss_dict_obj:
                    self[key]/=div_val[key]
                else:
                    self[key]/=div_val

        def _sum(self):
            total=0
            for val in self.values():
                total+=val
            return total

        def copy_keys(self,c_dict,keys):
            for key in keys:
                self[key]=c_dict[key]

        def apply(self,func):
            for key in self.keys():
                self[key]=self.tf2np(func(self[key]))

def get_best_model_path(exp_dir = 'logs/exp-D101'):
    paths = glob.glob(f'{exp_dir}/model/epoch_*')
    arg = np.argmin([float(path.split('_')[-1]) for path in paths])
    best_model_path = paths[arg] + '/weights'
    print("Found best model path : ",best_model_path)
    return best_model_path


def save_epoch_logs(loss_dict,epoch):
    if epoch==0: log_str=','.join(list(loss_dict.keys()))+'\n'
    else : log_str = ''
    log_str += ','.join(list(map(str,loss_dict.values())))+'\n'

    with open(os.path.join(TRAIN_CHECKPOINTS_FOLDER,'loss.csv'),'a+') as file:
        file.write(log_str)
            
def save_step_logs(train_loss,all_logs,epoch):
    if epoch==0 : log_str='epoch,'+','.join(list(train_loss.keys()))+'\n'
    else : log_str = ''
            
    for _ind in range(no_train_batch): 
        log_str += str(epoch)+','
        log_str += ','.join(list(map(str,all_logs[_ind])))+'\n'

    with open(os.path.join(TRAIN_CHECKPOINTS_FOLDER,'all_loss.csv'),'a+') as file:
        file.write(log_str)
        
def save_sample_losses(sample_losses,epoch):
    
    for _lind,loss_name in zip([0,1],['det','seg']):
        if TRAIN_LOSS_WTS[_lind]:
            if epoch==0 : log_str='epoch,'+','.join(list(map(str,range(TRAIN_BATCH_SIZE))))+'\n'
            else : log_str = ''
            for _ind in range(no_train_batch): 
                log_str += str(epoch)+','
                log_str += ','.join(list(map(str,sample_losses[_ind][_lind].numpy())))+'\n'

            with open(os.path.join(TRAIN_CHECKPOINTS_FOLDER,'sample_loss_'+loss_name+'.csv'),'a+') as file:
                file.write(log_str)  


@tf.function
def train_step(image_data, target,epoch,alpha=1.0):
        #tf.reset_dafault_graph()
        train_loss_dict = loss_dict_obj()
        giou_loss = conf_loss = prob_loss = 0.0
        yolo_model.training = True
        with tf.GradientTape(persistent=True) as tape:
            tape.watch([yolo_model.trainable_variables])
            pred_result = yolo_model(image_data)
            pred_result = yolo.decode_output(pred_result)
            del image_data
            
            if TRAIN_LOSS_WTS[0]:
                for i in range(NO_OF_GRID):
                    conv, pred = pred_result[i*2], pred_result[i*2+1]
                    #tf.print(pred.shape,conv.shape,target[i][0].shape)
                    loss_dict, _giou_loss , _conf_loss , _prob_loss = calc_yolo_loss(pred, conv, *target[i], i)
                    train_loss_dict.sum_update(loss_dict)
                    
                    giou_loss += 1/NO_OF_GRID * _giou_loss
                    conf_loss += 1/NO_OF_GRID * _conf_loss
                    prob_loss += 1/NO_OF_GRID * _prob_loss
                    
                loss =  (giou_loss + conf_loss + prob_loss)
                grads = tape.gradient(loss , yolo_model.trainable_variables)
                #grads = tape.gradient(MTL_LOSS_WTS[2] * prob_loss , yolo_model.trainable_variables)
                
        return train_loss_dict,grads

def reduce_prob(x):
    mean = tf.math.reduce_mean(x,axis=0)
    #std  = tf.math.reduce_std(x,axis=0)
    var  = tf.reduce_mean(tf.square(x - mean),axis=0) + 1e-20
    p_x  = tf.square((x - mean)) / var
    p_x  = tf.exp(-0.5*p_x)
    coef = 1/(tf.sqrt(2*np.pi)*var)
    p_x  = coef * p_x
    return (p_x/tf.reduce_sum(p_x,axis=0)) + 1e-20

def reduce_kl_div(x,y):
    p_x = reduce_prob(x)
    p_y = reduce_prob(y)
    kl_div = p_x * tf.math.log(p_x / p_y)
    kl_div = tf.reduce_sum(kl_div,axis=0)
    kl_div = tf.clip_by_value(kl_div,0,1)
    return kl_div

def get_probs(_grads,_grads_conf_1):
    coef_1 = 1/_vars
    coef_2 = tf.square(_grads - _means)/_vars
    probs  = coef_1 * tf.exp(-0.5*coef_2)
    probs  = (probs / tf.reduce_max(probs)) + 1e-10
    return probs

def get_disc_probs(_grads,_confs):
    sign      = np.sign(_confs)
    pos_prob  = np.mean((sign + 1.0)/2.0,axis=0)
    neg_prob  = np.mean(np.abs(sign - 1.0)/2.0,axis=0)
    pos_mask  = np.uint8(_grads>=0)
    neg_mask  = np.uint8(_grads<0)
    
    _grads    = pos_mask * pos_prob + neg_mask * neg_prob
    return _grads

def reduce_disc_kl_div(p_x,p_y):
    kl_div = p_x * tf.math.log(p_x / p_y)
    kl_div = tf.reduce_sum(kl_div,axis=0)
    kl_div = tf.clip_by_value(kl_div,0,1)
    return kl_div

#@tf.function    
def train_batch(image_data,target,frozen=False):
    NO_MINI_BATCH = TRAIN_BATCH_SIZE//TRAIN_MINI_BATCH_SIZE
    gradients = [0.0]*len(yolo_model.trainable_variables)
    train_loss = loss_dict_obj(); smp_loss = loss_dict_obj()
    
    for ind in range(0,NO_MINI_BATCH):
        mbatch_image,mbatch_target = get_mini_batch(image_data,target,ind,TRAIN_MINI_BATCH_SIZE)
        mbatch_train_loss, mbatch_grads = train_step(mbatch_image, mbatch_target,epoch)
        
        _coef = 1/NO_MINI_BATCH
        for ind,lgrad1 in enumerate(mbatch_grads):           
            if type(lgrad1) == type(None) : lgrad1 = 0.0
            gradients[ind] += _coef * lgrad1
                   
        train_loss.sum_update(mbatch_train_loss)
    train_loss.divide(NO_MINI_BATCH)    
    optimizer.apply_gradients(zip(gradients, yolo_model.trainable_variables))
        
    return train_loss

def add_ext(loss_dict,_ext='val_'):
    _loss_dict = {}
    for key in loss_dict.keys():
          _loss_dict[_ext+key]=loss_dict[key]
    return _loss_dict

#@tf.function
def validate_step(image_data, target):
    val_loss_dict = loss_dict_obj()
    
    pred_result = yolo_model(image_data)
    pred_result = yolo.decode_output(pred_result,batch_size=TEST_BATCH_SIZE)
    del image_data

    for i in range(NO_OF_GRID):
        conv, pred = pred_result[i*2], pred_result[i*2+1]
        loss_dict, _, _,_ = calc_yolo_loss(pred, conv, *target[i], i,batch_size=TEST_BATCH_SIZE)
        loss_dict = add_ext(loss_dict,_ext='val_')
        val_loss_dict.sum_update(loss_dict)

    if TRAIN_USE_DST or TRAIN_USE_SEG:
        seg_pred     = pred_result[2*(grid -1)+2:]
        seg_label    = target[3:]

        loss_dict, _, _,_ = calc_seg_loss(seg_label,seg_pred)
        loss_dict = add_ext(loss_dict,_ext='val_')
        val_loss_dict.sum_update(loss_dict)

    del pred_result, target
        
    return val_loss_dict

In [4]:
if len(TRAIN_LRES_PRE_WTS):
    yolo = YoloModel(training=True,input_size=TRAIN_HRES_INPUT_SIZE)
    yolo_model = yolo.get_model()
    model_path = get_best_model_path(exp_dir=TRAIN_LRES_PRE_WTS)
    yolo_model.load_weights(model_path)
    
    trainset = Dataset('train',input_size=TRAIN_HRES_INPUT_SIZE)
    testset  = Dataset('test',input_size=TRAIN_HRES_INPUT_SIZE)

else:
    yolo = YoloModel(training=True)
    yolo_model = yolo.get_model()
    trainset = Dataset('train')
    testset  = Dataset('test')

steps_per_epoch = len(trainset)

optimizer  = tf.keras.optimizers.Adam(lr = TRAIN_LR)
#loss_opt   =  tf.keras.optimizers.Adam(lr = TRAIN_LOSS_WTS_LR)
best_val_loss = 10e8 # should be large at start
no_train_batch = trainset.num_batchs
no_val_batch   = testset.num_batchs
yolo_model.summary()

mobilenet


2022-10-11 09:33:43.235003: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-10-11 09:33:43.235165: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-11 09:33:43.235312: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1660 Ti computeCapability: 7.5
coreClock: 1.59GHz coreCount: 24 deviceMemorySize: 5.80GiB deviceMemoryBandwidth: 268.26GiB/s
2022-10-11 09:33:43.235345: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-10-11 09:33:43.235363: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-10-11 09:33:43.235374: I tensorflow/stream_executor/platfo

getting wts from Conv1  === setting weights from Conv1 to block0_conv1
getting wts from bn_Conv1  === setting weights from bn_Conv1 to block0_bn_conv1
getting wts from expanded_conv_depthwise  === setting weights from expanded_conv_depthwise to block_xx_expanded_conv_depthwise
getting wts from expanded_conv_depthwise_BN  === setting weights from expanded_conv_depthwise_BN to block_xx_expanded_conv_depthwise_BN
getting wts from expanded_conv_project  === setting weights from expanded_conv_project to block_xx_expanded_conv_project
getting wts from expanded_conv_project_BN  === setting weights from expanded_conv_project_BN to block_xx_expanded_conv_project_BN
getting wts from block_1_expand  === setting weights from block_1_expand to block_1_expand
getting wts from block_1_expand_BN  === setting weights from block_1_expand_BN to block_1_expand_BN
getting wts from block_1_depthwise  === setting weights from block_1_depthwise to block_1_depthwise
getting wts from block_1_depthwise_BN  === s

In [5]:
if DEBUG_MODE == True:
    no_train_batch = 20
    no_val_batch   = 3
    TRAIN_EPOCHS   = 20

In [6]:
if DATA_GEN == True:
    print(len(trainset),len(testset))
    
    os.system(f"rm -r {TRAIN_DATA_SAVE_PATH}*")
    for ind in trange(no_train_batch):
        train_img, train_label  = next(trainset)
        path=TRAIN_DATA_SAVE_PATH+'batch_{}.npy'.format(str(ind))
        for _i in range(NO_OF_GRID):
            train_label[_i][0] = zlib.compress(train_label[_i][0])
        np.save(path,[np.uint8(train_img*255.0),train_label]) 
        
    os.system(f"rm -r {TEST_DATA_SAVE_PATH}*")
    for ind in trange(no_val_batch):
        test_img, test_label = next(testset)
        path=TEST_DATA_SAVE_PATH+'batch_{}.npy'.format(str(ind))
        for _i in range(NO_OF_GRID):
            test_label[_i][0] = zlib.compress(test_label[_i][0])
        np.save(path,[np.uint8(test_img*255.0),test_label])
        
    del trainset, testset

1849 1250


  0%|                                                  | 0/1849 [00:00<?, ?it/s]/home/irfan/Desktop/Mlops-Env/python_38/lib/python3.8/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
100%|███████████████████████████████████████| 1250/1250 [00:46<00:00, 26.93it/s]


In [7]:
def get_mini_batch(image,target,ind,mbatch_size):
    ind1,ind2 = ind*mbatch_size,(ind+1)*mbatch_size
    mbatch_target  = [[elem[ind1:ind2] for elem in target[0]],\
                      [elem[ind1:ind2] for elem in target[1]],\
                      [elem[ind1:ind2] for elem in target[2]]]
    mbatch_target += [elem[ind1:ind2] for elem in target[3:]]
    return image[ind1:ind2], mbatch_target

if not os.path.exists(TRAIN_CHECKPOINTS_FOLDER): os.makedirs(TRAIN_CHECKPOINTS_FOLDER)
os.system(f'mkdir -p {TRAIN_CHECKPOINTS_FOLDER}/model')
    
epoch = -1
learning_rates = np.linspace(TRAIN_LR/10,TRAIN_LR,TRAIN_WARM_UP_EPOCHS*no_train_batch)
_frozen= TRAIN_FREEZE_EPOCH>0

OUTPUT_SIZES = TRAIN_INPUT_SIZE // np.array(YOLO_STRIDES)
def decompress(x,shape=(16,14,14,3,85)):
    x = zlib.decompress(x)
    x = np.frombuffer(x,dtype=np.float32)
    x = x.reshape(shape)
    return x

while 1:
    epoch +=1
    if os.path.exists('params.json'):
        with open('params.json','r') as file:
            param_dict = json.load(file)
        globals().update(param_dict)
    
    if _frozen==True and epoch>=TRAIN_FREEZE_EPOCH:
        for ind,layer in enumerate(yolo_model.layers):   
            if 'block' in layer.name:
                yolo_model.layers[ind].trainable = True
            print(ind,layer.name,layer.trainable)
            yolo_model.summary()
            _frozen=False
            
    if epoch >= TRAIN_EPOCHS:
        break
 
    all_logs = [] ; sample_losses =[]
    train_loss_dict=loss_dict_obj(); val_loss_dict = loss_dict_obj(); loss_dict={'Epoch' : epoch}
    
    batch_seg=[];batch_dst=[]
    for batch_ind in tqdm(range(no_train_batch)):
        
        tstep = (epoch)*no_train_batch + batch_ind
        if tstep < len(learning_rates):
            optimizer.learning_rate = learning_rates[tstep]
            
        if TRAIN_INPUT_SIZE <TRAIN_SAVE_THR_SIZE :
            path=TRAIN_DATA_SAVE_PATH+'batch_{}.npy'.format(str(batch_ind))
            batch_data = np.load(path,allow_pickle=True)
            batch_data[0] = np.float32(batch_data[0]/255.0)
            for _i in range(NO_OF_GRID):
                shape = (TRAIN_BATCH_SIZE,OUTPUT_SIZES[_i],OUTPUT_SIZES[_i],3,NUM_CLASS+5)
                batch_data[1][_i][0] = decompress(batch_data[1][_i][0],shape)
               
        else:
            batch_data=next(trainset)
        
        image_data, target = batch_data[0],list(batch_data[1])+[batch_dst,batch_seg]
        train_loss = train_batch(image_data, target,frozen=_frozen)
        del image_data, target, batch_data
        
        train_loss_dict.sum_update(train_loss)
        all_logs+=[list(train_loss.values())]
        
    train_loss_dict.divide(no_train_batch)
    iou_val, conf_val, prob_val, total_val = 0, 0, 0, 0
    batch_seg=[];batch_dst=[]
    
    for batch_ind in tqdm(range(no_val_batch)):
        if TRAIN_INPUT_SIZE < TRAIN_SAVE_THR_SIZE:
            path=TEST_DATA_SAVE_PATH+'batch_{}.npy'.format(str(batch_ind))
            batch_data=np.load(path,allow_pickle=True)
            batch_data[0] = np.float32(batch_data[0]/255.0)
            for _i in range(NO_OF_GRID):
                shape = (TEST_BATCH_SIZE,OUTPUT_SIZES[_i],OUTPUT_SIZES[_i],3,NUM_CLASS+5)
                batch_data[1][_i][0] = decompress(batch_data[1][_i][0],shape)
        else:
            batch_data = next(testset)
        
        image_data, target = batch_data[0],list(batch_data[1])+[batch_dst,batch_seg]
        val_loss = validate_step(image_data, target)
        del image_data, target, batch_data
        val_loss_dict.sum_update(val_loss)
        
    val_loss_dict.divide(no_val_batch)
    loss_dict.update(train_loss_dict)
    loss_dict.update(val_loss_dict)
    print(loss_dict)
    
     
    if epoch % TRAIN_SAVE_WEIGHTS_EVERY == 0:
        save_directory = os.path.join(TRAIN_CHECKPOINTS_FOLDER,'model', 'epoch_{}_val_det_loss_{}'.format(epoch,round(loss_dict['val_det_loss'],4)))
        yolo_model.save_weights(save_directory+'/weights')
        
    if TRAIN_SAVE_BEST_ONLY and best_val_loss>loss_dict['val_det_loss']:
        save_directory = os.path.join(TRAIN_CHECKPOINTS_FOLDER, 'model')
        yolo_model.save_weights(save_directory+'/weights')
        best_val_loss = loss_dict['val_det_loss']

    save_epoch_logs(loss_dict,epoch)
    save_step_logs(train_loss,all_logs,epoch)
    #save_sample_losses(sample_losses,epoch)

  0%|                                                  | 0/1849 [00:00<?, ?it/s]

2022-10-11 09:53:34.222935: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-10-11 09:53:34.363965: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2599990000 Hz
2022-10-11 09:53:36.427796: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2022-10-11 09:53:43.788975: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2022-10-11 09:53:43.857351: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2022-10-11 09:53:44.915172: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-10-11 09:53:44.916598: I tensorflow/stream

{'Epoch': 0, 'iou_loss': 8.139648252593433, 'conf_loss': 6.526475961302473, 'prob_loss': 23.28584636153886, 'det_loss': 37.95197056666653, 'val_iou_loss': 7.9210609375, 'val_conf_loss': 5.623584375, 'val_prob_loss': 17.9423234375, 'val_det_loss': 31.486965625}
0 block0_conv1 True
Model: "mobile_net_v2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block0_conv1 (Conv2D)        multiple                  864       
_________________________________________________________________
block0_bn_conv1 (BatchNormal multiple                  128       
_________________________________________________________________
block0_conv1_relu (ReLU)     multiple                  0         
_________________________________________________________________
inv_res_block_0 (_inverted_r multiple                  992       
_________________________________________________________________
inv_res_block_1 (_inverted_r multipl

100%|███████████████████████████████████████| 1250/1250 [01:32<00:00, 13.58it/s]


{'Epoch': 1, 'iou_loss': 7.711472078166568, 'conf_loss': 5.337517106771598, 'prob_loss': 16.274687909899434, 'det_loss': 29.32367709960855, 'val_iou_loss': 7.71947734375, 'val_conf_loss': 5.20347421875, 'val_prob_loss': 15.51823125, 'val_det_loss': 28.441178125}


...

{'Epoch': 22, 'iou_loss': 7.019153560929327, 'conf_loss': 4.253190715226565, 'prob_loss': 6.745912347115846, 'det_loss': 18.018256636681976, 'val_iou_loss': 7.32604609375, 'val_conf_loss': 4.839326171875, 'val_prob_loss': 16.4993328125, 'val_det_loss': 28.664715625}


100%|███████████████████████████████████████| 1250/1250 [01:32<00:00, 13.49it/s]


{'Epoch': 23, 'iou_loss': 7.009701122781146, 'conf_loss': 4.241081294335179, 'prob_loss': 6.5816206531050145, 'det_loss': 17.832403063064913, 'val_iou_loss': 7.331521875, 'val_conf_loss': 4.883453515625, 'val_prob_loss': 17.0549296875, 'val_det_loss': 29.269934375}


100%|███████████████████████████████████████| 1250/1250 [01:47<00:00, 11.68it/s]


{'Epoch': 24, 'iou_loss': 7.0102109242414, 'conf_loss': 4.237453650822183, 'prob_loss': 6.441341571771757, 'det_loss': 17.689006120466168, 'val_iou_loss': 7.43835234375, 'val_conf_loss': 4.938259765625, 'val_prob_loss': 17.0882984375, 'val_det_loss': 29.464859375}
